In [1]:
import os
from openai import OpenAI


In [2]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential



In [3]:
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    ToolMessage,
    AssistantMessage
)

In [4]:
models = {
    "DeepSeek-R1": "DeepSeek-R1",
    "Phi-4-mini-instruct": "Phi-4-mini-instruct",
    "gpt-4o": "gpt-4o",
    "o3-mini": "o3-mini"
}

In [5]:
endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT")

In [6]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(os.getenv("AZURE_INFERENCE_SDK_ENDPOINT_KEY")),
    credential_scopes=["https://cognitiveservices.azure.com/.default"],

    )

In [7]:
messages = [
    UserMessage(content="1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?"),
]

# DeepSeek-R1

In [8]:
response = client.complete(
    messages = messages,
    model = models['DeepSeek-R1']
)



In [9]:
print(response.choices[0].message.content)

<think>
Okay, let me try to figure out this problem. So, the question is: if one shirt takes 1 hour to dry, how long does it take to dry 3 shirts? Hmm, at first glance, it seems straightforward, but I need to make sure I'm not missing anything here.

First, let me parse the question again. The key information given is that one shirt needs 1 hour to dry. Now, the question is asking about drying three shirts. The obvious answer might be that it still takes 1 hour because each shirt dries independently. But maybe there's a trick here. Sometimes these problems test whether you assume things that aren't stated. For example, maybe if you have more shirts, they take longer because they are drying together in the same space, blocking airflow or something. But the problem doesn't mention any constraints like that. It just says 1 shirt takes 1 hour. So, unless told otherwise, we probably shouldn't assume that the drying time increases with the number of shirts.

Wait, but let me think again. If 

In [10]:
import re

match = re.match(r"<think>(.*?)</think>(.*)", response.choices[0].message.content, re.DOTALL)

print("Response:", )
if match:
    print("\tThinking:", match.group(1))
    print("\tAnswer:", match.group(2))
else:
    print("\tAnswer:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tTotal tokens:", response.usage.total_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)

Response:
	Thinking: 
Okay, let me try to figure out this problem. So, the question is: if one shirt takes 1 hour to dry, how long does it take to dry 3 shirts? Hmm, at first glance, it seems straightforward, but I need to make sure I'm not missing anything here.

First, let me parse the question again. The key information given is that one shirt needs 1 hour to dry. Now, the question is asking about drying three shirts. The obvious answer might be that it still takes 1 hour because each shirt dries independently. But maybe there's a trick here. Sometimes these problems test whether you assume things that aren't stated. For example, maybe if you have more shirts, they take longer because they are drying together in the same space, blocking airflow or something. But the problem doesn't mention any constraints like that. It just says 1 shirt takes 1 hour. So, unless told otherwise, we probably shouldn't assume that the drying time increases with the number of shirts.

Wait, but let me th

# gpt-4o

In [11]:
print(messages[0].content)
# gpt-4o
response = client.complete(
    messages = messages,
    model = models["gpt-4o"]
)
print(response.choices[0].message.content)

1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?
If **1 shirt takes 1 hour to dry**, and you are drying **multiple shirts at the same time**, the drying time remains **1 hour** because they can dry simultaneously.

So, for drying **3 shirts**, it will still take **1 hour** assuming you are drying them under the same conditions together.


# Phi-4-mini-instruct

In [12]:
response = client.complete(
    messages = messages,
    model = models['Phi-4-mini-instruct']
)
print(response.choices[0].message.content)

It would take 3 hours to dry 3 shirts, assuming each shirt still requires 1 hour to dry and they can be dried independently of each other. However, if the drying process for all shirts must happen sequentially (one after the other), then it would still take 3 hours, since you can't dry all shirts simultaneously if each shirt requires its own hour.


# o3-mini

calling o3-mini is not supported with the current version of azure.ai.inference SDK

In [13]:
from openai import AzureOpenAI

In [14]:
aoai_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview")


In [15]:
chat_prompt = chat_prompt = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": messages[0].content
            }
        ]
    }]

In [16]:
completion = aoai_client.chat.completions.create(
    model="o3-mini",
    messages=messages,
    max_completion_tokens=100000,
    stop=None,
    stream=False ,

    reasoning_effort="high"  )

In [17]:
print(completion.choices[0].message.content)

If you dry the shirts at the same time (for example, hanging them on a drying rack with enough space), then each shirt drys independently. This means 3 shirts will dry in 1 hour—the same time it takes 1 shirt to dry.


In [18]:
print(completion.to_json())

{
  "id": "chatcmpl-BDk6uE04rM1zZ5eoxweNmjPVjPalw",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "If you dry the shirts at the same time (for example, hanging them on a drying rack with enough space), then each shirt drys independently. This means 3 shirts will dry in 1 hour—the same time it takes 1 shirt to dry.",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1742615212,
  "model": "o3-mini-2025-01-31",
  "object": "chat.completion",
  "system_fingerprint": "fp_ded0d1